# Import Required Libraraies

In [1]:
import re
import numpy as np
import pandas as pd

# Read Data

In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Purification and Feature Engineering

## Name
### Extract titles from names

In [4]:
train_data['Title'] = train_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))
test_data['Title'] = test_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

# Display the unique titles
print(f'All unique titles in names (Train): {train_data["Title"].unique()}')
print(f'All unique titles in names (Test): {test_data["Title"].unique()}')

All unique titles in names (Train): ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']
All unique titles in names (Test): ['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [5]:
# Train Data
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

# Test Data
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')

In [6]:
print(f'Train Titles: {train_data["Title"].unique()}')
print(f'Test Titles: {test_data["Title"].unique()}')

Train Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']
Test Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']


### Drop Names

In [7]:
train_data = train_data.drop('Name', axis=1)
test_data = test_data.drop('Name', axis=1)

## Ticket
### Drop ticket columns

In [8]:
# Drop Ticket Column
train_data = train_data.drop('Ticket', axis=1)
test_data = test_data.drop('Ticket', axis=1)

## Sex
### Convert sexes to 0s and 1s

In [9]:
# Encode Sex
train_data['Sex'] = train_data['Sex'].replace({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].replace({'male': 0, 'female': 1})
print(f'Number of distinct values for sex: {set(test_data["Sex"])}')

Number of distinct values for sex: {0, 1}


## Cabin
### Extract letters from Cabins

In [10]:
train_data['Cabin'] = train_data['Cabin'].fillna("X")
print(train_data['Cabin'].str[0].unique())

test_data['Cabin'] = test_data['Cabin'].fillna("X")
print(test_data['Cabin'].str[0].unique())

['X' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']
['X' 'B' 'E' 'A' 'C' 'D' 'F' 'G']


### Encode letters

In [11]:
cabin_to_number = {'X': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}

train_data['Cabin_Encoded'] = train_data['Cabin'].apply(lambda x: cabin_to_number[x[0]])
test_data['Cabin_Encoded'] = test_data['Cabin'].apply(lambda x: cabin_to_number[x[0]])

### Drop Cabin Column

In [12]:
train_data = train_data.drop('Cabin', axis=1)
test_data = test_data.drop('Cabin', axis=1)

## Embarked
### Fill Null values with 'S'

In [13]:
train_data['Embarked'].describe()

train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

### Encode Embarked Values

In [14]:
embark_to_number = {'S': 0, 'C': 1, 'Q': 2}

train_data['Embarked'] = train_data['Embarked'].apply(lambda x: embark_to_number[x[0]])
test_data['Embarked'] = test_data['Embarked'].apply(lambda x: embark_to_number[x[0]])

In [15]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_Encoded
0,1,0,3,0,22.0,1,0,7.2500,0,Mr,0
1,2,1,1,1,38.0,1,0,71.2833,1,Mrs,3
2,3,1,3,1,26.0,0,0,7.9250,0,Miss,0
3,4,1,1,1,35.0,1,0,53.1000,0,Mrs,3
4,5,0,3,0,35.0,0,0,8.0500,0,Mr,0


In [16]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_Encoded
0,892,3,0,34.5,0,0,7.8292,2,Mr,0
1,893,3,1,47.0,1,0,7.0000,0,Mrs,0
2,894,2,0,62.0,0,0,9.6875,2,Mr,0
3,895,3,0,27.0,0,0,8.6625,0,Mr,0
4,896,3,1,22.0,1,1,12.2875,0,Mrs,0
